# Rotina para Acompanhamento Macroeconômico

## Importando as bibliotecas utilizadas

In [ ]:
import pandas as pd
import requests

## Função de extração das tabelas por url

In [ ]:
def retorna_tabelas(url: str) -> list:
    try:
        url = url

        header = {
            "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
            "X-Requested-With": "XMLHttpRequest"
        }

        r = requests.get(url, headers = header)
        tabelas = pd.read_html(r.text)
        return tabelas
    
    except HTTPError as e:
        print("The server returned an HTTP error")
    
    except URLError as e:
        print("The server could not be found!")

## Commodities

In [ ]:
def tabela_commodities(tabela_commodities: list, df: pd.DataFrame) -> pd.DataFrame:
    for i in range(0, 5):
        commodities = tabela_commodities[i].dropna(how = 'all', axis = 1)
        commodities = commodities.rename(columns = {'Unnamed: 3' : 'Var', 
                                                    f'{commodities.columns[0]}' : 'Commodities'})
        commodities = commodities.set_index(commodities.columns[0])
        df = pd.concat([df, commodities])
    
    df.rename_axis('Commodities', inplace = True)
    
    return df

### Preços Atuais

In [ ]:
url_commodities_atuais = 'https://tradingeconomics.com/commodities'
tabela_commodities_atuais = retorna_tabelas(url_commodities_atuais)

In [ ]:
commodities_atuais = pd.DataFrame(columns = ['Price', 'Var', 'Day', 'Weekly', 'Monthly', 'YTD', 'Date'])
commodities_atuais = tabela_commodities(tabela_commodities_atuais, commodities_atuais)

In [ ]:
commodities_atuais

#### Salvando em excel

In [ ]:
commodities_atuais.to_excel('Commodities_Atuais.xls', encoding = 'UTF-8')

### Preços Futuros

In [ ]:
url_commodities_futuras = 'https://tradingeconomics.com/forecast/commodity'
tabela_commodities_futuras = retorna_tabelas(url_commodities_futuras)

In [ ]:
commodities_futuras = pd.DataFrame(columns = ['Price', 'Var', 'Day'])
commodities_futuras = tabela_commodities(tabela_commodities_futuras, commodities_futuras)

In [ ]:
commodities_futuras

#### Salvando em excel

In [ ]:
commodities_futuras.to_excel('Commodities_Futuras.xls', encoding = 'UTF-8')

## Índices Macroeconômicos

In [ ]:
url_indices_macro = 'https://tradingeconomics.com/'
indices_macro = retorna_tabelas(url_indices_macro)

In [ ]:
indices_macro = indices_macro[-1]
indices_macro = indices_macro.rename(columns = {indices_macro.columns[0] : 'Países'})
indices_macro = indices_macro.set_index('Países')

In [ ]:
indices_macro

#### Salvando em excel

In [ ]:
indices_macro.to_excel('Índices_Macro.xls', encoding = 'UTF-8')

## Índices Bolsas Mundiais

In [ ]:
def trata_indices(df: pd.DataFrame) -> pd.DataFrame:
    df.index = df.index.map(lambda x: x[:-len('derived')] if 'derived' in x else x)
    return df

### Atuais

In [ ]:
url_indices_atuais = 'https://www.investing.com/indices/major-indices'
tabelas_indices_atuais = retorna_tabelas(url_indices_atuais)

In [ ]:
indices_mundo_atuais = tabelas_indices_atuais[0][['Index', 'Last', 'High', 'Low', 'Chg.', 'Chg. %']].set_index('Index')
indices_mundo_atuais = trata_indices(indices_mundo_atuais)
indices_mundo_atuais.head()

#### Salvando em excel

In [ ]:
indices_mundo_atuais.to_excel('Indices_Mundo_Atuais.xls', encoding = 'UTF-8')

### Futuros

In [ ]:
url_indices_futuros = 'https://br.investing.com/indices/indices-futures'
tabelas_indices_futuros = retorna_tabelas(url_indices_futuros)

In [ ]:
indices_mundo_futuros = tabelas_indices_futuros[0][['Índice', 'Mês', 'Último', 
                                                   'Máxima', 'Mínima', 'Var. %', 'Hora']].set_index('Índice')
indices_mundo_futuros = trata_indices(indices_mundo_futuros)
indices_mundo_futuros.head()

#### Salvando em excel

In [ ]:
indices_mundo_futuros.to_excel('Indices_Mundo_Futuros.xls', encoding = 'UTF-8')